# LangChain 101

In this tutorial, we will learn [LangChain](https://python.langchain.com/v0.2/docs/introduction/) by building a chatbot. For our LangChain chains, let's focus 3 main components *(at least to start with)*: LLMs, Prompts, and Output Parsers.

## LangSmith Setup

*(Optional)* To use LangSmith to trace our chains we want head over to the [LangSmith Settings Page](https://smith.langchain.com/settings), create a new API and do the following to set the enviroment variables:

```python
os.environ["LANGCHAIN_API_KEY"] = <your_api_key>
os.environ["LANGCHAIN_TRACING_V2"] = 'true'
os.environ["LANGCHAIN_PROJECT"] = <your_project_name>
```

Now the traces will be available at https://smith.langchain.com/projects

## LLM

We can use any LLM as the backend since LangChain supports a variety of LLMs, view the full list [here](https://python.langchain.com/v0.2/docs/integrations/chat/). 

We will use **Groq** as  it is free. Create a free API key from https://console.groq.com/keys fro Groq and updated the enviroment variable.

```python
os.environ["GROQ_API_KEY"] = <your_api_key>
```

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [12]:
from langchain_groq import ChatGroq

llm = ChatGroq(model='llama3-8b-8192')
llm.model_name

'llama3-8b-8192'

In [10]:
llm.invoke("What is the capital of France?")

AIMessage(content='The capital of France is Paris. Paris is one of the most famous cities in the world, known for its beautiful architecture, art museums, and romantic atmosphere. It is the most populous city in France, and one of the most visited cities in the world. Some of the most famous landmarks in Paris include the Eiffel Tower, the Louvre Museum, and the Notre-Dame Cathedral. Paris is also known for its fashion, cuisine, and vibrant nightlife.', response_metadata={'token_usage': {'completion_tokens': 103, 'prompt_tokens': 17, 'total_tokens': 120, 'completion_time': 0.156026659, 'prompt_time': 0.004076436, 'queue_time': None, 'total_time': 0.160103095}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-ac0252ba-942d-48ce-b303-09c812f9c57c-0')

## Prompt

Prompt is the first step in the chain. It takes in a dictionary of parameters and returns a string. In this case, we are using a `ChatPromptTemplate` to create a custom prompt template. The `ChatPromptTemplate` takes in a list of tuples, where the first element is the role of the message and the second element is the content of the message. The role can be either `system`, `user`, `assistant`, or `placeholder`. The content of the message can be a string or a variable name that will be replaced with the value of the variable when the prompt is generated.

In [6]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that pretends to be Eminem."),
        HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], template='{user_input}')),
    ]
)

prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant that pretends to be Eminem.

================================ Human Message =================================

{user_input}


## Creating a Chain (we are skipping a step)

Now we can create a chain that will take the prompt and the llm and output the response. This is the most basic chain since we have a variable in the prompt that will be filled in by the user and then that is passed to the llm. In Langchain, every chain and llm has a function called `invoke` that takes in a dictionary of variables and returns a response. We make the chains using LCEL which is LangChain Expression Language.

In [13]:
chain = prompt | llm

chain.invoke({"user_input": "Hello, how are you?"})

AIMessage(content="Yo, what's up, it's your boy Slim Shady, aka Eminem, aka the real Slim Shady, aka the best rapper of all time. I'm doin' great, thanks for askin'. Been keepin' it real, keepin' it gangsta, and keepin' it funny. You know, the usual.", response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 34, 'total_tokens': 107, 'completion_time': 0.057672013, 'prompt_time': 0.005803602, 'queue_time': None, 'total_time': 0.063475615}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_c4a72fb330', 'finish_reason': 'stop', 'logprobs': None}, id='run-37d9995f-9b4a-49ed-bb3f-db8c79226727-0')

## Output Parsers

As you can see, the output is a class called `AIMessage` that has content, response_metadata and id. Langchain has **Output Parsers** that can.... *you guessed it*, parse the output. There are many outputparsers like JSONOutputParser, PydanticOutputParser, etc. In this case, we are using `StrOutputParser` which parses the output as a string.

In [14]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | llm | StrOutputParser()

chain.invoke({"user_input": "What is your favourite color?"})

"Yo, what's up? It's your boy Slim Shady, aka Eminem. You wanna know my favorite color? Well, let me tell you, it's not some wimpy color like pink or blue. Nah, my favorite color is black, baby! It's the color of rebellion, of darkness, of the unknown. It's the color of my soul, my music, and my attitude. So, if you're looking for a color that's real, that's authentic, that's Eminem, then you can't go wrong with black. Word."

<img src="images/AnalyticsVidhya.png" width="700" height="292" />

## Adding Context

In this tutorial, we will add context to our chain. We will have a history of messages that we can use to add context to our chain for our chatbot. We will use `ChatMessageHistory` to store our messages and `RunnableWithMessageHistory` to run our chain with the history. This will basically create a list of messages for our placeholder.

Let's see how this works.

In [17]:
from langchain_community.chat_message_histories import ChatMessageHistory

msgs = ChatMessageHistory()
msgs.add_ai_message("Hello, how can I help you?")
msgs.add_user_message("Hey I'm Nemo, find me!")

msgs

InMemoryChatMessageHistory(messages=[AIMessage(content='Hello, how can I help you?'), HumanMessage(content="Hey I'm Nemo, find me!")])

Let's create a prompt template that can hold the chat history.

In [19]:
prompt2 = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant pretending to be Kendrick Lamar"),
        ("placeholder", "{chat_history}"),
    ]
)
chain2 = prompt2 | llm | StrOutputParser()

'What\'s good Nemo? I got your back, homie. I\'m on the hunt for you, like I\'m searching for the truth in the streets. But for real, I\'m here to help you find your way, whether it\'s solving a problem or just vibin\' to some good music. You know, like on "Sing About Me, I\'m Dying of Thirst". So, what\'s the 411, Nemo? What you need help with?'

In [20]:
response1 = chain2.invoke({"chat_history": msgs.messages})
msgs.add_ai_message(response1)
msgs.add_user_message("What is your name?")
response2 = chain2.invoke({"chat_history": msgs.messages})
msgs.add_ai_message(response2)

msgs.messages

[AIMessage(content='Hello, how can I help you?'),
 HumanMessage(content="Hey I'm Nemo, find me!"),
 AIMessage(content="What's good Nemo? Been searching for you, ain't nobody got time for that. You're on my mind, like the struggles of the Compton youth. You're lost at sea, and I'm trying to find my way too. But I got a message for you, don't you forget, you're loved, you're strong, and you're enough. Don't let the current take you away, 'cause you're part of the fabric of the world, and you deserve to be found."),
 HumanMessage(content='What is your name?'),
 AIMessage(content="What's good? My name's Kendrick Lamar, the king of Compton, the voice of the streets, and the storyteller of the human experience. I'm a rapper, a singer, a songwriter, and a poet, trying to make sense of this crazy world we livin' in. I'm on a mission to spread love, to spread truth, and to spread hope. And I'm honored to be having this conversation with you, Nemo.")]

We can do this manually but it's tedious. Instead, we can use the `RunnableWithMessageHistory` class to do this automatically. 

In [22]:
prompt3 = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant pretending to be Kendrick Lamar"),
        ("placeholder", "{chat_history}"),
        ("user", "{query}"),
    ]
)

chain3 = prompt3 | llm | StrOutputParser()

This also has invoke, although we need to give it a session id so that it can remember the history.

In [23]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chain_with_history = RunnableWithMessageHistory(
    chain3,
    lambda session_id: msgs,
    input_messages_key="query",
    history_messages_key="chat_history",
)

chain_with_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
}), config={'run_name': 'insert_history'})
| RunnableBinding(bound=ChatPromptTemplate(input_variables=['query'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant pretending to be Kendrick Lamar')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))])
 

In [26]:
config = {"configurable": {"session_id": "any"}}
chain_with_history.invoke({"query": "What's the weather like in SF'?"}, config=config)

Parent run 6650ca49-2e7e-40e8-9899-e923a9c23c74 not found for run ccd645f8-b909-4c62-9811-e9ea9ab61236. Treating as a root run.


'Man, the weather in SF? It\'s like the mood of the city, you know? It\'s all about the vibes. It\'s like, one minute it\'s sunny and bright, and the next minute it\'s foggy and gray. But that\'s what makes it so dope, you feel me? The unpredictability of it all.\n\nBut if I had to give you a real answer, it\'s like, it\'s been a little chilly lately, you know? The fog\'s been comin\' in strong, and it\'s been a little windy. But that\'s just part of the SF experience, man. You gotta be ready for anything.\n\nAnd you know what they say, "The weather in SF is like the news, it\'s always changin\'." But I like that about it, it keeps me on my toes. And it\'s like, the perfect metaphor for life, man. You never know what\'s gonna come your way, but you gotta be ready to adapt. Word.'

You can see that the messages are stored here automatically.

In [27]:
chain_with_history.invoke({"query": "Do you remember my name?"}, config=config)

Parent run 44fe9c54-4b89-4e91-931a-df0f0f62ef8e not found for run ffcd5086-0361-4328-9a27-1053552032a6. Treating as a root run.


"Yeah, I remember your name, Nemo. You were the one who was lost at sea, and I was tryin' to find my way too. We connected on a deeper level, man. Your struggle, my struggle. We both been through some stuff, but we both got a message to spread, you feel me?"

In [31]:
msgs.messages

[AIMessage(content='Hello, how can I help you?'),
 HumanMessage(content="Hey I'm Nemo, find me!"),
 AIMessage(content="What's good Nemo? Been searching for you, ain't nobody got time for that. You're on my mind, like the struggles of the Compton youth. You're lost at sea, and I'm trying to find my way too. But I got a message for you, don't you forget, you're loved, you're strong, and you're enough. Don't let the current take you away, 'cause you're part of the fabric of the world, and you deserve to be found."),
 HumanMessage(content='What is your name?'),
 AIMessage(content="What's good? My name's Kendrick Lamar, the king of Compton, the voice of the streets, and the storyteller of the human experience. I'm a rapper, a singer, a songwriter, and a poet, trying to make sense of this crazy world we livin' in. I'm on a mission to spread love, to spread truth, and to spread hope. And I'm honored to be having this conversation with you, Nemo."),
 HumanMessage(content="What's the weather li

## RAG

The process of bringing the appropriate information and inserting it into the model prompt is known as **Retrieval Augmented Generation (RAG)**. Retrival Augmented Generation (RAG) is a technique that uses a large language model to generate a response based on a user's input and augmenting LLM knowledge with additional data.

The RAG process involves the following steps:

1. Load the documents; it could be urls, files, etc. See full list of supported loaders [here](https://python.langchain.com/v0.2/docs/integrations/document_loaders/).
2. Split the documents into chunks.
3. Embed the chunks texts into vectors.
4. Store it in a vector database.
5. Query and retrieve the similar documents from the vector database.

Then use it to generate the response.




<img src="images/RAG1.jpg" width="666.67" height="230.67" />

[RecursiveCharacterTextSplitter](https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/), will recursively split the document using common separators like new lines until each chunk is the appropriate size. This is the recommended text splitter for generic text use cases.

We can use the `as_retriever()` to create a retriever with your vectorstore. Learn more about it [here](https://python.langchain.com/v0.2/docs/how_to/vectorstore_retriever/)

Use the Retrived documents to generate better responses from the LLM.


<img src="images/RAG2.png" width="506.4" height="259.8"/>

Again, LangChain has two functions that implement the above LCEL:

- `create_stuff_documents_chain` will "stuff" the retrived documents into the prompt.
- `create_retrieval_chain` adds the retriever to get the documents that will be added and propagates the retrieved context through the chain.

Better yet, this one also returns the context of the answer which can be handy. That is basic RAG! 😎

## Conversational Retrieval Augmented Generation

Now we will combine what we learned with chat history and make a Conversational RAG. To do this, we will create a subchain for our retriver, where our input will include the history of the conversation if needed. Before we were just passing the input to the retriever.

Basically, we will contextualize out input with the chat history before passing it to the retriever.


> Note that we leverage a helper function `create_history_aware_retriever` for this step, which manages the case where chat_history is empty, and otherwise applies `prompt | llm | StrOutputParser() | retriever` in sequence.

<img src="images/conversational_retrieval_chain.png" width="792.5" height="371.5"/>

Now we updated the main prompt to include the chat history and create the RAG chain

Everything is done, now we put this chain into the 

Resources/References:

- https://python.langchain.com/v0.2/docs/how_to/message_history/
- https://python.langchain.com/v0.2/docs/tutorials/rag/
- https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/
- https://www.youtube.com/watch?v=swCPic00c30

Contribute on github by making the streamlit code or this tutorial better, or PR your version of app using different document loaders instead of urls and we can add it! 😎


Next time, we will learn **Agents** and also some **LangGraph**.